In [1]:
import Pkg;
Pkg.add("ModalIntervalArithmetic")
Pkg.add("LinearAlgebra")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
using ModalIntervalArithmetic
using LinearAlgebra

pos(x) = x > 0 ? x : 0
neg(x) = x < 0 ? -x : 0

function sti(𝐱)
    n = size(𝐱, 1)
    x = zeros(2 * n)
    for i = 1:n
        x[i] = -𝐱[i].inf
        x[i+n] = 𝐱[i].sup
    end
    return x
end

function sti_inv(x)
    n = size(x, 1) ÷ 2
    𝐱 = zeros(ModalInterval, n)
    for i = 1:n
        𝐱[i] = ModalInterval(-x[i], x[i+n])
    end
    return 𝐱
end

function ∂x_pos(x)
    if x > 0
        return 1
    elseif x == 0
        return 0.5 # actually something ∈ [0, 1]
    else
        return 0
    end
end

function ∂x_neg(x)
    if x < 0
        return -1
    elseif x == 0
        return -0.5 # actually something ∈ [-1, 0]
    else
        return 0
    end
end

function ∂max_1(C, i, j, x)
    n = size(x, 1) ÷ 2
    prod_1 = pos(C[i, j].sup)pos(x[j])
    prod_2 = neg(C[i, j].inf)pos(x[j+n])
    if prod_1 > prod_2
        return (pos(C[i, j].sup), 0)
    elseif prod_1 == prod_2
        return (0.5pos(C[i, j].sup), 0.5neg(C[i, j].inf))
    else
        return (0, neg(C[i, j].inf))
    end
end

function ∂max_2(C, i, j, x)
    n = size(x, 1) ÷ 2
    prod_1 = pos(C[i, j].sup)pos(x[j+n])
    prod_2 = neg(C[i, j].inf)pos(x[j])
    if prod_1 > prod_2
        return (0, pos(C[i, j].sup))
    elseif prod_1 == prod_2
        return (0.5neg(C[i, j].inf), 0.5pos(C[i, j].sup))
    else
        return (neg(C[i, j].inf), 0)
    end
end

function ∂F(C, i, x)
    n = size(x, 1) ÷ 2
    res = zeros(2 * n)
    if 1 <= i <= n
        for j = 1:n
            temp = ∂max_1(C, i, j, x)
            res_1 = pos(C[i, j].inf)∂x_neg(x[j]) + neg(C[i, j].sup)∂x_neg(x[j+n]) - temp[1]
            res_2 = pos(C[i, j].inf)∂x_neg(x[j]) + neg(C[i, j].sup)∂x_neg(x[j+n]) - temp[2]
            res[j] -= res_1
            res[j+n] -= res_2
        end
    else
        i -= n
        for j = 1:n
            temp = ∂max_2(C, i, j, x)
            res_1 = temp[1] - pos(C[i, j].inf)∂x_neg(x[j+n]) - neg(C[i, j].sup)∂x_neg(x[j])
            res_2 = temp[2] - pos(C[i, j].inf)∂x_neg(x[j+n]) - neg(C[i, j].sup)∂x_neg(x[j])
            res[j] += res_1
            res[j+n] += res_2
        end
    end
    return res
end

function D(C, x)
    n = size(x, 1)
    D = zeros(n, n)
    for i = 1:n
        D[i, :] = ∂F(C, i, x)
    end
    return D
end

function init_point(C, d)
    midC = mid.(C)
    C̃ = [pos.(midC) neg.(midC)
         neg.(midC) pos.(midC)]
    return C̃ \ sti(d)
end

function sub_diff(C, d, x0, eps)
    𝒢(x) = sti(C * sti_inv(x)) - sti(d)

    x = x0
    𝒢_val = 𝒢(x)
    count = 0

    while(norm(𝒢_val) >= eps)
        println("x ", x)
        try
            x -= inv(D(C, x)) * 𝒢_val
        catch;
            println("Subgradient D is singular")
            break
        end
        𝒢_val = 𝒢(x)
        count += 1
    end

    return (sti_inv(x), count)
end


A = [ModalInterval(2.0, 4.0) ModalInterval(-2.0, 1.0)
         ModalInterval(1.0, 2.0) ModalInterval(2.0, 4.0)]
b = [ModalInterval(-2.0, 2.0), ModalInterval(-2.0, 2.0)]

x0 = init_point(A, b)


println("x0    = ", x0)
(x, count) = sub_diff(A, b, x0, 1.e-6)
println("x     = ", x)
println("count = ", count)



    

x0    = [0.6060606060606061, 0.3636363636363636, 0.6060606060606061, 0.3636363636363637]
x 

[0.6060606060606061, 0.3636363636363636, 0.6060606060606061, 0.3636363636363637]
x     = ModalInterval[[-0.3333333333333333, 0.33333333333333337], [-0.33333333333333337, 0.3333333333333333]]
count = 1
